In [1]:
import pandas as pd
import MetaTrader5 as mt5
from datetime import datetime,timedelta
import time
import talib as ta

In [2]:
#tz = pytz.timezone('America/Sao_Paulo')
ativo = 'EURUSD'
#ativos = ['EURUSD']

path = r'C:\Program Files\MT5 by FOREX.com Terminal\terminal64.exe'

if not mt5.initialize(path = path):
    print("initialize() failed")
    mt5.shutdown()

In [3]:

def comprar(symbol,stop,hora_fechamento):
    point = mt5.symbol_info(symbol).point
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": 0.1,
        "type": mt5.ORDER_TYPE_BUY,
        "price": mt5.symbol_info_tick(symbol).ask,
        "sl": stop,
        "tp": 0.0,
        "deviation": 20,
        "magic": 234000,
        "comment": "python script open",
        "type_time": mt5.ORDER_TIME_SPECIFIED,
        "expiration": hora_fechamento,
        "type_filling": mt5.ORDER_FILLING_RETURN,
    }
    result = mt5.order_send(request)
    print(result)

def vender(symbol,stop,hora_fechamento):
    point = mt5.symbol_info(symbol).point
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": 0.1,
        "type": mt5.ORDER_TYPE_SELL,
        "price": mt5.symbol_info_tick(symbol).bid,
        "sl": stop,
        "tp": 0.0,
        "deviation": 20,
        "magic": 234000,
        "comment": "python script open",
        "type_time": mt5.ORDER_TIME_SPECIFIED,
        "expiration": hora_fechamento,
        "type_filling": mt5.ORDER_FILLING_RETURN,
    }
    result = mt5.order_send(request)
    print(result)

In [15]:
def verificar(ativo, hora_fechamento):

    df = pd.DataFrame(mt5.copy_rates_from_pos(ativo, mt5.TIMEFRAME_M1, 0, 200))
    df.columns = ['Data','Open','High','Low','Close','Vol','','']
    df['MA_9'] = ta.EMA(df['Close'], timeperiod = 9)
    df['MA_9_diff_value'] = df['MA_9'].diff(periods = 1)
    df['MA_9_tend'] = df['MA_9_diff_value'].apply(lambda x: 1 if x > 0 else -1)
    df['MA_9_tend_diff'] = df['MA_9_tend'].diff()
    df['MA_100'] = ta.SMA(df['Close'],timeperiod=100)
    df['tend'] = df.apply(lambda x: 1 if x['Close'] > x['MA_100'] else 0, axis = 1)

    df['acao'] = df.apply(lambda x: 'sell' if (x['MA_9_tend_diff'] == 2 and x['tend'] == 1) 
        else 'call' if  (x['MA_9_tend_diff'] == -2 and x['tend'] == 0)  else 0, axis =1)

    df['preco'] = df.apply(lambda x: x['Close'], axis =1)

    df['stop'] = df.apply(lambda x: x['High'] + (x['High'] - x['Low']) * 2  if x['acao']=='sell' else x['Low'] - (x['High'] - x['Low']) * 2 if x['acao'] == 'call' else 0, axis =1)

    
    ultima_linha = df.tail(1)
    preco_entrada = ultima_linha['preco'].values[0]
    stop = ultima_linha['stop'].values[0]
    acao = ultima_linha['acao'].values[0]
    hora_fechamento = hora_fechamento

    if acao == 'call':
        comprar(ativo, stop,hora_fechamento)
    elif acao == 'sell':
        vender(ativo, stop,hora_fechamento)
    else:
        print(f'Não houve operação:')



In [16]:
while True:
    data_e_hora_atuais = datetime.now()
    hora_fechamento = data_e_hora_atuais + timedelta(minutes = 20)
    hora_fechamento = int(hora_fechamento.timestamp())
    hora_teste = str(data_e_hora_atuais.second)  
    if hora_teste == '59':
        verificar(ativo, hora_fechamento)
        #print(str(data_e_hora_atuais))
        time.sleep(10)

OrderSendResult(retcode=10027, deal=0, order=0, volume=0.0, price=0.0, bid=0.0, ask=0.0, comment='AutoTrading disabled by client', request_id=0, retcode_external=0, request=TradeRequest(action=1, magic=234000, order=0, symbol='EURUSD', volume=0.1, price=1.0619, stoplimit=0.0, sl=1.0605000000000002, tp=0.0, deviation=20, type=0, type_filling=2, type_time=2, expiration=1672021379, comment='python script open', position=0, position_by=0))
Não houve operação:
Não houve operação:


KeyboardInterrupt: 

In [14]:
data_e_hora_atuais = datetime.now()
hora_fechamento = data_e_hora_atuais + timedelta(minutes = 5*60 + 1)
hora_fechamento = int(hora_fechamento.timestamp())
symbol = 'EURUSD'
request = {
    "action": mt5.TRADE_ACTION_DEAL,
    "symbol": symbol,
    "volume": 0.1,
    "type": mt5.ORDER_TYPE_BUY,
    "price": mt5.symbol_info_tick(symbol).ask,
    "sl": 0.0,
    "tp": 0.0,
    "deviation": 20,
    "magic": 234000,
    "comment": "python script open",
    "type_time": mt5.ORDER_TIME_SPECIFIED,
    "expiration": hora_fechamento,
    "type_filling": mt5.ORDER_FILLING_IOC,
}
result = mt5.order_send(request)
print(result)

OrderSendResult(retcode=10030, deal=0, order=0, volume=0.0, price=0.0, bid=0.0, ask=0.0, comment='Unsupported filling mode', request_id=0, retcode_external=0, request=TradeRequest(action=1, magic=234000, order=0, symbol='EURUSD', volume=0.1, price=1.06375, stoplimit=0.0, sl=0.0, tp=0.0, deviation=20, type=0, type_filling=1, type_time=2, expiration=1672108619, comment='python script open', position=0, position_by=0))


In [25]:
data_e_hora_atuais = datetime.now()
hora_fechamento = data_e_hora_atuais + timedelta(minutes = 0.5)
hora_fechamento = int(hora_fechamento.timestamp())

In [12]:
str(datetime.fromtimestamp(hora_fechamento))

'2022-12-26 23:40:14'

In [ ]:
datetime.from